In [ ]:
import json
import re
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xmltodict
from pydicom import dcmread
from scipy.ndimage import binary_fill_holes
from skimage.measure import regionprops
from skimage.morphology import label
from tqdm import tqdm_notebook

from utils.visual import plot_vol, plot_vol_with_overlay

In [ ]:
df = pd.read_csv('./csvs/nodules.csv')

In [ ]:
df.head()

,calcification,internalStructure,lobulation,malignancy,margin,noduleID,path,servicingRadiologistID,sphericity,spiculation,subtlety,texture
0,6,1,1,3,5,Nodule 001,y:LIDC-IDRI/LIDC-IDRI-0039/01-01-2000-49300/30...,299556309,4,1,4,5
1,5,1,1,1,5,Nodule 005,y:LIDC-IDRI/LIDC-IDRI-0039/01-01-2000-49300/30...,299556309,5,1,4,5
2,6,1,1,2,5,Nodule 002,y:LIDC-IDRI/LIDC-IDRI-0039/01-01-2000-49300/30...,299556309,5,1,3,5
3,3,1,1,1,5,Nodule 006,y:LIDC-IDRI/LIDC-IDRI-0039/01-01-2000-49300/30...,299556309,3,1,4,5
4,6,1,1,3,5,Nodule 004,y:LIDC-IDRI/LIDC-IDRI-0039/01-01-2000-49300/30...,299556309,4,1,4,5


In [ ]:
counts = df.path.value_counts()

In [ ]:
counts.head()

y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST WO CONTRAST-09860/2-82004                  49
y:LIDC-IDRI/LIDC-IDRI-0129/01-01-2000-CT ANGIO CHEST WWO C-30279/2-BOTTOM TO TOP-12518    48
y:LIDC-IDRI/LIDC-IDRI-0855/01-01-2000-13854/5476-ThoraxRoutine  3.0  B31f-57503           40
y:LIDC-IDRI/LIDC-IDRI-0583/01-01-2000-18747/30077-81802                                   40
y:LIDC-IDRI/LIDC-IDRI-0049/01-01-2000-35347/3000627-75427                                 39
Name: path, dtype: int64

In [ ]:
def get_metadata(path):
    dcms = [dcmread(path) for path in glob(path+'/*.dcm')]
    dcms.sort(key=lambda x: float(x.ImagePositionPatient[2]))
    slice_thichness = abs(float(dcms[1].ImagePositionPatient[2]) - float(dcms[0].ImagePositionPatient[2]))
    first_slice_location = float(dcms[0].ImagePositionPatient[2])
    n_slices = len(dcms)
    return slice_thichness, first_slice_location, n_slices, dcms

def fill_holes(arr):
    # se verifica si es un volumen o un corte
    if len(arr.shape) == 3:
        filled_arr = []
        # en caso de ser un volumen se rellenan los agujeroscorte a corte
        for slice_ in arr:
            filled_arr.append(fill_holes(slice_))
        return np.array(filled_arr)
    elif len(arr.shape) == 2:
        # en el caso de ser un corte
        # primero se conviente a una mascara binaria ya que el metodo 'binary_fill_holes' solo funciona para mascaras binarias
        binary_arr = arr != 0
        # se aplica el metodo 'binary_fill_holes' a el corte
        binary_arr = binary_fill_holes(binary_arr).astype(np.int)
        # se etiquetan las regiones conectadas con el metodo 'label'
        labeled_arr = label(binary_arr)
        values = np.unique(labeled_arr)
        values = values[values != 0]
        for value in values:
            real_values = arr[labeled_arr == value]
            binary_arr[labeled_arr == value] = real_values.max()
        return binary_arr
    
def preprocess_roi_volumes(roi_volumes):
    if not isinstance(roi_volumes, np.ndarray):
        roi_volumes = np.array(roi_volumes)
        
    binary_roi_volumes = roi_volumes != 0
    binary_roi_volume = binary_roi_volumes.any(axis=0)
    labeled_roi_volume = label(binary_roi_volume)
    
    last_label = np.max(labeled_roi_volume)
    for roi_volume in roi_volumes:
        used_values = []
        values = np.unique(roi_volume)
        values = values[values != 0]
        for value in values:
            temp_label = np.unique(labeled_roi_volume[roi_volume == value])
            temp_label = temp_label[0]
            if temp_label in used_values:
                last_label += 1
                labeled_roi_volume[roi_volume == value] = last_label
                temp_label = last_label
            used_values.append(temp_label)

    sum_roi_volumes = binary_roi_volumes.sum(axis=0)
    
    grouped_nodules = {}
    for region in regionprops(labeled_roi_volume):
        grouped_nodules[region.label] = {}
        cords = region.bbox
        grouped_nodules[region.label]['zi'] = cords[0]
        grouped_nodules[region.label]['zf'] = cords[3]
        
        grouped_nodules[region.label]['yi'] = cords[1]
        grouped_nodules[region.label]['yf'] = cords[4]
        
        grouped_nodules[region.label]['xi'] = cords[2]
        grouped_nodules[region.label]['xf'] = cords[5]
        
        grouped_nodules[region.label]['own_ids'] = []
        
        
    for roi_volume in roi_volumes:
        for value in grouped_nodules.keys():
            values, counts = np.unique(roi_volume[labeled_roi_volume == value], return_counts=True)
            if len(values) == 1 and values[0] == 0:
                continue
            counts = counts[values != 0]
            values = values[values != 0]
            own_id = values[np.argmax(counts)]
            if own_id != 0:
                grouped_nodules[value]['own_ids'].append(own_id)
        
    return grouped_nodules, binary_roi_volume

In [ ]:
# generador de IDs finales para los nodulos
final_id_gen = iter(range(1, 10000))

data_df = pd.DataFrame(columns=['path', 'noduleID', 'own_id', 'final_id'])
bboxes = []

# se recore cada path de los archivos csv
for path in tqdm_notebook(counts.index, total=len(counts)):
# for path in ['y:LIDC-IDRI/LIDC-IDRI-0436/01-01-2000-51874/5348-ChestRoutine  3.0  B31f-55181']:
    # lee el archivo
    with open(glob(path + '/*.xml')[0], "r") as file:
        text = file.read()
    
    # extrae del path el identificador de caso LIDC-IDRI (son los 4 numeros al final del cada carpeta)
    lidc_name = 'LIDC-IDRI-' + re.search('LIDC-IDRI-(.+?)/', path).group(1)
    
    # se conviente de texto en formato xml a json
    parsed_text = xmltodict.parse(text)
    info = json.loads(json.dumps(parsed_text))
    
    if 'LidcReadMessage' in list(info.keys()):
        info = info['LidcReadMessage']
    else:
        info = info['IdriReadMessage']
        
    # se extrae de los dicoms informacion necesaria para generar el roi
    slice_thichness, first_slice_location, n_slices, dcms = get_metadata(path)
    
    # se almacenará en 'roi_volumes' los cuatro rois de los radiologos
    roi_volumes = []
    # en 'all_data' se almacenara el id de los nodulos el path y el id propio temporal
    all_data = []
    # generador de IDs temporales para los nodulos
    id_gen = iter(range(1, 200))
    
    # se recorre cada session (hay almenos 4 por xml)
    for session in info['readingSession']:
        # roi generado a partir de las anotaciones
        roi_volume = np.zeros((n_slices, 512, 512), dtype=np.int)
        
        # si este radiologo no anotó nodulos se salta a la siguiente sesion
        if 'unblindedReadNodule' not in session.keys():
            continue
        # si si solo hay un nodulo, se convierte a una lista para evitar errors ya que el codigo itera sobre una lista
        if not isinstance(session['unblindedReadNodule'], list):
            session['unblindedReadNodule'] = [session['unblindedReadNodule']]
    
        # se recorre cada session no ciega
        for unblindedReadNodule in session['unblindedReadNodule']:
            data = {}
            # se almacenan algunos atributos para luego ser comparados
            data["path"] = path
            data['noduleID'] = unblindedReadNodule['noduleID']
            
            # se valida si es un nodulo de tipo 1
             # se valida si hay mas de un roi ya que significaría que está en mas de un corte y por lo tanto es un nodulo tipo 1
            if not isinstance(unblindedReadNodule['roi'], list):
                # si solo hay un roi se valida si este tiene mas de un punto, de lo contrario se pasa al siguiente nodulo
                if not isinstance(unblindedReadNodule['roi']['edgeMap'], list):
                    continue
                unblindedReadNodule['roi'] = [unblindedReadNodule['roi']]
                
            # id temporal para el nodulo
            own_id = next(id_gen)
            data['own_id'] = own_id
            all_data.append(data)
            
            # se recorre cada roi que pueda tener un nodulo, tiene mas de uno ya que los rois se hacen por corte
            for roi in unblindedReadNodule['roi']:
                # el sliceLocation donde se encuentra el roi
                n_slice = float(roi['imageZposition'])
                # se convierte de sliceLocation a una coordenada para el volumen roi
                z = int((n_slice - first_slice_location) / slice_thichness)
                
                # se valida si es inclusivo, esto quiere decir que el roi hace parte del nodulo
                # de lo contrario significa que no hace parte del nodulo
                is_inclusive = roi['inclusion'] == 'TRUE'
                
                # se valida si "roi['edgeMap']" es un arreglo
                if not isinstance(roi['edgeMap'], list):
                    roi['edgeMap'] = [roi['edgeMap']]
                    
                # se recorre cada punto del roi
                for edgeMap in roi['edgeMap']:
                    x = int(edgeMap['xCoord'])
                    y = int(edgeMap['yCoord'])
                    # en el roi se setea con el id temporal o cero dependiendo si el roi es inclusivo
                    roi_volume[z, y, x] = np.int(own_id) if is_inclusive else np.int(0)
        # ya que en el dataset lo que proveen es un margen, se rellana cada agujero
        roi_volume = fill_holes(roi_volume)
        roi_volumes.append(roi_volume)           
    all_data = pd.DataFrame(all_data)
    
    # se procesan los rois para saber cuales nodulos anotados son los mismos
    # 'grouped_nodules' contiene listas con los ids de los nodulos que son los mismos
    # 'binary_roi_volume' contiene la intercepcion de los rois
    grouped_nodules, binary_roi_volume = preprocess_roi_volumes(roi_volumes)
    
    temp = []
    # se recorren los grupos de ids
    for group in grouped_nodules:
        # grupo de ids
        own_ids = grouped_nodules[group]['own_ids']
        # se obtiene un id unico (consecutivo)
        final_id = next(final_id_gen)
        # se setea en el dataframe el grupo de ids(id temporal || mismo nodulo) con un unico id
        all_data.loc[all_data.own_id.isin(own_ids), 'final_id'] = final_id
        # elimino el grupo de ids temporales
        temp.append(grouped_nodules[group]['own_ids'])
        del grouped_nodules[group]['own_ids']
        # añado la informacion de los bounding boxes junto con el unico id
        bboxes.append({'final_id': final_id, **grouped_nodules[group]})
    # se va concatenando 'all_data' a 'data_df' para almacenar toda la informacion
    data_df = pd.concat([data_df, all_data], ignore_index=True, sort=True)
    # se almacena la mascara binaria
    np.savez_compressed('y:LIDC-IDRI_final_masks/{}.npz'.format(lidc_name), mask=binary_roi_volume.astype(np.uint8))
        
# se crea el dataframe con los bounding boxes
bboxes = pd.DataFrame(bboxes)

In [ ]:
# almacenando csv
# bboxes.to_csv('./csvs/bboxes.csv')
# data_df.to_csv('./csvs/news_ids_and_old_ids.csv')

In [ ]:
# un registro no tiene id
# se hizo una verificacion manual y es debido a que el mismo radiologo marcó 2 veces el mismo nodulo
data_df.final_id.isna().sum()

1

In [ ]:
data_df[data_df.final_id.isna()]

,Unnamed: 0,final_id,noduleID,own_id,path
5366,5366,NaN,3,5,y:LIDC-IDRI/LIDC-IDRI-0436/01-01-2000-51874/53...


In [ ]:
data_df = data_df[~data_df.final_id.isna()]

In [ ]:
data_df = data_df[['final_id', 'noduleID', 'path']]
data_df.head()

,final_id,noduleID,path
0,5.0,Nodule 001,y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST...
1,6.0,Nodule 005,y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST...
2,12.0,Nodule 006,y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST...
3,15.0,Nodule 016,y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST...
4,16.0,Nodule 012,y:LIDC-IDRI/LIDC-IDRI-0686/01-01-2000-CT CHEST...


In [ ]:
# dft = df.merge(data_df, left_on=['noduleID', 'path'], right_on=['noduleID', 'path'])
dft = data_df[['final_id', 'noduleID']]

In [ ]:
dft.head()

,final_id,noduleID
0,5.0,Nodule 001
1,6.0,Nodule 005
2,12.0,Nodule 006
3,15.0,Nodule 016
4,16.0,Nodule 012


In [ ]:
count_by_id = dft.groupby('final_id').count()

In [ ]:
count_by_id.head()

,noduleID
final_id,
1.0,2
2.0,1
3.0,1
4.0,3
5.0,4


In [ ]:
for i in range(1, 5):
    print('con un concenso de {} radiologo(s) la cantidad de nodulos es {}'.format(i, (count_by_id >= i).noduleID.sum()))

con un concenso de 1 radiologo(s) la cantidad de nodulos es 2686
con un concenso de 2 radiologo(s) la cantidad de nodulos es 1885
con un concenso de 3 radiologo(s) la cantidad de nodulos es 1388
con un concenso de 4 radiologo(s) la cantidad de nodulos es 901
